In [3]:
'''
Load scripts
'''

%run -i '../scripts/py/config.py'
%run -i '../scripts/py/obj_det.py'
# %run -i '../scripts/py/visualization_utils.py'

/Users/jessedecker/miniconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jessedecker/miniconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jessedecker/miniconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/jessedecker/miniconda3/envs/tf

### Object Detection

1. Select Model
2. Load Labelmap
3. Set Parameters
4. Run Detection

In [55]:
'''
Select Model

# Default is "SSD with Mobilenet". 
# See the detection model zoo (https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) 
for a list of other models that can be run out-of-the-box with varying speeds and accuracies.
'''

# Set model name
# MODEL_NAME = 'ssd_mobilenet_v1_coco_2018_01_28'
MODEL_NAME = 'faster_rcnn_inception_v2_coco_2018_01_28'

# Set model folder location
model_folder = GetModelsFolder()+"/"+MODEL_NAME
# print(model_folder)

# Load a (frozen) Tensorflow model into memory for detection.
graph = 'frozen_inference_graph.pb'
PATH_TO_CKPT = model_folder+'/'+graph

# PATH_TO_CKPT = '/Users/jessedecker/projects/air_monitor/resources/vehicle_counting_tensorflow-master/ssd_mobilenet_v1_coco_2018_01_28/frozen_inference_graph.pb'
# print(PATH_TO_CKPT)

/Users/jessedecker/projects/air_monitor/resources/vehicle_counting_tensorflow-master/ssd_mobilenet_v1_coco_2018_01_28/frozen_inference_graph.pb


In [56]:
'''
Load labelmap
'''

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = GetDetectionFolder()+'/'+'mscoco_label_map.pbtxt'
NUM_CLASSES = 14

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map,max_num_classes=NUM_CLASSES, use_display_name=True)

category_index = label_map_util.create_category_index(categories)
print(category_index)

{1: {'id': 1, 'name': 'person'}, 2: {'id': 2, 'name': 'bicycle'}, 3: {'id': 3, 'name': 'car'}, 4: {'id': 4, 'name': 'motorcycle'}, 5: {'id': 5, 'name': 'airplane'}, 6: {'id': 6, 'name': 'bus'}, 7: {'id': 7, 'name': 'train'}, 8: {'id': 8, 'name': 'truck'}, 9: {'id': 9, 'name': 'boat'}, 10: {'id': 10, 'name': 'traffic light'}, 11: {'id': 11, 'name': 'fire hydrant'}, 13: {'id': 13, 'name': 'stop sign'}, 14: {'id': 14, 'name': 'parking meter'}}


In [57]:
'''
Set start params and perform detection
'''

month = 10
days = [11]
hours = [11]
calls = [1,2,3,4]

# cams = [498,499,507,508,509,510,511,512,513,514,855,856,857,858]
cams = [498]

targets = [2,3,4,5,6,7,8,9]
threshold = 0.4

for day in days:
    for hour in hours:
        StartDetection(month,day,hour,calls,cams,targets,threshold)

Starting session


### Visualize Detections

In [15]:
'''
  Adds a bounding box to an image.
  Bounding box coordinates can be specified in either absolute (pixel) or
  normalized coordinates by setting the use_normalized_coordinates argument.
'''

def DrawBoxes(image,
              boxes,
              classes,
              color='black',
              thickness=2):
    
    draw = ImageDraw.Draw(image)
    
    i=0
    for box in boxes:
        left = box[0]
        right = box[1]
        top = box[2]
        bottom = box[3]
        
#         print(classes[i])
        if classes[i] == 'car':
            color = 'blue'
        elif classes[i] == 'truck':
            color = 'green'
        elif classes[i] == 'bus':
            color = 'red'
        i+=1
            
        draw.line([(left, top), (left, bottom), (right, bottom),
                   (right, top), (left, top)], width=thickness, fill=color)
    
    image.show()

In [14]:
'''
Load Hourly CSV
'''

month = 10
day = 11
hour = 11

filename = GetImageRoot(month,day,hour)+'/'+'image_results.csv'
# print(GetImageRoot(month,day,hour))

images = pd.read_csv(filename)
images = images.sort_values('ImagePath')

images.head()

,ImagePath,class,score,left,right,top,bottom
0,../data/cameras/M10/D11/H11/C498/498_1.jpg,car,0.732821,202,231,115,134
1,../data/cameras/M10/D11/H11/C498/498_1.jpg,car,0.695173,217,241,111,133
2,../data/cameras/M10/D11/H11/C498/498_1.jpg,car,0.650648,40,72,153,170
3,../data/cameras/M10/D11/H11/C498/498_1.jpg,car,0.572687,119,170,169,201
4,../data/cameras/M10/D11/H11/C498/498_2.jpg,car,0.764545,196,220,147,164


In [16]:
'''
Call to draw images with boxes
'''

paths = set(images.ImagePath.tolist()[:])
for path in paths:
    tmp = images[images['ImagePath']==path]
    
    vh_classes = tmp['class'].tolist()
    
    rows = tmp.index.tolist()
    boxes = []
    for row in rows:
        boxes.append(images.loc[row,'left':])
        
    img = Image.open(images.iloc[row]['ImagePath'])
    DrawBoxes(img,boxes,vh_classes)

In [86]:
'''
Create a label map
'''

with open('../data/detection/map.csv') as f:
    file = f.read().split('\n')
    
# print(file)
# print(tmp)
# file = pd.read_csv()
# categories = file.read()
end = '\n'
s = ' '
class_map = {}
for ID, name in enumerate(file):
#     print(ID,name)
    out = ''
    out += 'item' + s + '{' + end
    out += s*2 + 'id:' + ' ' + (str(ID+1)) + end
    out += s*2 + 'name:' + ' ' + '\'' + name + '\'' + end
    out += '}' + end*2
    

    with open('../data/detection/map.pbtxt', 'a') as f:
        f.write(out)
        
    class_map[name] = ID+1

In [ ]:
# Download Model if needed
# MODEL_FILE = MODEL_NAME + '.tar.gz'
# DOWNLOAD_BASE = \
#     'http://download.tensorflow.org/models/object_detection/'